# Abuse Detection using BERT COLAB

In [2]:
# Install Google Drive related libraries
!pip install PyDrive

In [ ]:
# install ktrain on Google Colab
!pip3 install ktrain

In [4]:
# Gooble Drive related libraries
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# General libraries
import pandas as pd
import numpy as np

# import speficic libraries
import tensorflow as tf
import ktrain 
from ktrain import text

In [5]:
# Authenticate Google Drive Use
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
# Download file from Google Drive to Colab local machine
# When a file in Google Drive is shared with link we get an ID in the link. We use this ID to download the file.
downloaded = drive.CreateFile({'id':"1L8_qOpO9e5ECr8e_TN49o9gTR94zh_8E"})   # replace the id with id of file you want to access
downloaded.GetContentFile('LearningLossModelQMI_5.csv')  # replace the file name with your file in Google Drive

In [7]:
downloaded = drive.CreateFile({'id':"1F-oAazFo01PPXlCk3qo6y_Djf0Gf21Tq"})   # replace the id with id of file you want to access
downloaded.GetContentFile('QmiTest.csv')  # replace the file name with your file in Google Drive

In [8]:
# Load training file from COLAB local machine
df_train = pd.read_csv('LearningLossModelQMI_5.csv')
df_train.head(5)

,text,label
0,my dad is a whore we were supposed to leave to...,0
1,xatp he did you a favor then . don ’ t need no...,1
2,xatp kill yourself you disgusting whore of a d...,1
3,xatp xatp do you feel hurt by slut shaming ! s...,0
4,"`` i have those shoes ! take my shoes off , wh...",0


In [9]:
# Load testing file from COLAB local machine
df_test = pd.read_csv('QmiTest.csv')
df_test.head(5)

,label,text
0,1,@ winegirl73 2 bit attention seeking whore
1,0, i was dodging bullets in slow motion you slut 
2,0,@ mayoroflondon sadiq khan-proud mayor of the ...
3,0,"@ lewdpon3 i agree , you should definitely lea..."
4,1,@ peoples_pundit @ fluffydogattack @ sengillib...


In [10]:
import re
import html

class CleanTweets:
  '''
  This class has some simple data cleaning methods. 
  '''
    def __init__(self):
        self.BOS = 'xbos'  # beginning-of-sentence tag
        self.FLD = 'xfld'  # data field tag
        
        self.XNEL = 'xnel' #non english occurence
        self.XURL = 'xurl' #url occurence
        self.XATP = 'xatp' #@Person occurance
        self.XRTU = 'xrtu:' #retweet unmodified
        self.XRTM = 'xrtm:' #retweet modified
        
        self.re1 = re.compile(r'  +')
        
    def cleanNonAscii(text):
        '''
        Remove Non ASCII characters from the dataset. Naive way to remove non english
        Arguments:
            text: str
        returns: 
            text: str
        '''
        return ''.join(i for i in text if ord(i) < 128)
        
    
    
    def prepro_a_tweet(self, tweet):
        
        def fixup(x):
            """ Cleans up erroroneus characters"""
            x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
                'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
                '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
                ' @-@ ','-').replace('\\', ' \\ ').replace('\n', ' ').replace('\t', ' ').replace('\r', ' ').replace('rt @','@')
            return self.re1.sub(' ', html.unescape(x))
        
        tweet = fixup(tweet)
        tweet = re.sub(r'^RT @\w+:', self.XRTU, tweet)
        tweet = re.sub(r'^MRT @\w+:', self.XRTM, tweet)
        tweet = re.sub(r'@\w+', self.XATP, tweet)
        tweet = re.sub(r'http\S+', self.XURL, tweet)
        tweet = re.sub(r'(.)\1+', r'\1\1', tweet).strip()
        tweet = tweet.lower()
        return tweet

ct = CleanTweets()

In [11]:
# Apply simple data cleaning to training set
df_train['text'] = df_train['text'].apply(ct.prepro_a_tweet)
df_train.head(5)

In [ ]:
# Apply simple data cleaning to testing set
df_test['text'] = df_test['text'].apply(ct.prepro_a_tweet)
df_test.head(5)

In [12]:
TEXT_FIELD = 'text'
LABEL_FIELD = 'label'

In [13]:
# Seperate data to independent (X) and dependent (y) variables
X_train, y_train = df_train[TEXT_FIELD].to_list(), df_train[LABEL_FIELD].to_list()
X_test, y_test = df_test[TEXT_FIELD].to_list(), df_test[LABEL_FIELD].to_list()

In [14]:
# Prepare data to model useable format and download pretrained model BERT
(xtrain,  ytrain), (xtest, ytest), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=[0, 1],
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [15]:
# Creat the text classifier model with pretrained model BERT and prepared training data
model = text.text_classifier(name='bert', train_data=(xtrain, ytrain), preproc=preproc)

Is Multi-Label? False
maxlen is 350
done.


In [16]:
# get learning to train the text classifer model
learner = ktrain.get_learner(model=model,
                             train_data = (xtrain, ytrain),
                             val_data = (xtest, ytest),
                             batch_size = 6)

In [17]:
# fit the model one cycle
learner.fit_onecycle(lr=2e-5, epochs=1)



begin training using onecycle policy with max lr of 2e-05...
500/500 [==============================] - 391s 783ms/step - loss: 0.4998 - accuracy: 0.7547 - val_loss: 0.5469 - val_accuracy: 0.7099


In [18]:
# creat a predictor to predict on unknown data
predictor = ktrain.get_predictor(learner.model, preproc)

In [19]:
predictor.get_classes()

[0, 1]

In [20]:
# predict on unknown data
predicted = predictor.predict(X_test)

In [21]:
# let's save the predictor for later use
predictor.save('/tmp/my_predictor')

In [22]:
# reload the predictor
reloaded_predictor = ktrain.load_predictor('/tmp/my_predictor')

In [23]:
# make a prediction on the same document to verify it still works
reloaded_predictor.predict(X_test[0:10])

[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]